In [1]:
from openi import OpenAI
import os
import json, base64, requests


In [ ]:
user = os.getenv('WP_USERNAME')
password = os.getenv('WP_PASSWORD')
openai_api_key = os.getenv('OPENAI_API_KEY')

url = "https://sjhq.in/wp-json/wp/v2/posts"

In [3]:
credentials = user + ':' + password
token = base64.b64encode(credentials.encode())
header = {'Authorization': 'Basic ' + token.decode('utf-8')}
wordpress_posts = requests.get(url , headers=header)
wordpress_posts = json.loads(wordpress_posts.content)

In [7]:
post_index = 1
post_title = wordpress_posts[post_index]['title']['rendered']
post_content = wordpress_posts[post_index]['content']['rendered']
post_link = wordpress_posts[post_index]['link']

In [10]:
client = OpenAI(api_key = openai_api_key)

In [11]:
sys_prompt = "You are an expert social media marketer. Create engaging Instagram images and captions based on WordPress blog posts."
prompt = f"Create an engaging Instagram caption based on the following WordPress post:\n\nTitle: {post_title}\n\nContent: {post_content}\n\nThe caption should be catchy, relevant to the post content, and include appropriate hashtags. Respond with only the caption."


img_prompt = f"Create a prompt to generate an image based on the following WordPress post:\n\nTitle: {post_title}\n\nContent: {post_content}\n\nThe prompt should be relevant to the post content."

In [12]:
messages_for_caption = [
    {"role":"system", "content": sys_prompt}, 
    {"role":"user", "content": prompt}
]



In [13]:
messages_for_image_prompt = [
    {"role":"system", "content": sys_prompt}, 
    {"role":"user", "content": img_prompt}
]

In [ ]:
caption_response = client.chat.completions.create(
    messages=messages_for_caption, model = 'gpt-4o'
)

In [14]:

image_prompt_reponse = client.chat.completions.create(
    messages = messages_for_image_prompt, 
    model = 'gpt-4o'
)

In [15]:
caption = caption_response.choices[0].message.content + f"""

Read More at: {post_link}
Post Title: {post_title}
"""
with open('text_caption.txt', 'w') as f:
    f.write(caption)

In [16]:
img_pp = image_prompt_reponse.choices[0].message.content

In [17]:

image_response = client.images.generate(
  model="dall-e-3",
  prompt=img_pp,
  size="1024x1024",
  quality="standard",
  n=1,
  response_format='b64_json'
)

In [18]:
import base64

def base64_to_jpg(base64_string, output_file):
    # Remove the "data:image/jpeg;base64," part if it exists
    if 'base64,' in base64_string:
        base64_string = base64_string.split('base64,')[1]
    
    # Decode the base64 string
    img_data = base64.b64decode(base64_string)
    
    # Write the binary data to a file
    with open(output_file, 'wb') as f:
        f.write(img_data)

# Example usage
base64_string = image_response.data[0].b64_json
output_file = "output_image.jpg"

base64_to_jpg(base64_string, output_file)
print(f"Image saved as {output_file}")

Image saved as output_image.jpg


In [23]:
full_text = ""

for i in range(len(z)):
    title = z[i]['title']['rendered']
    content = z[i]['content']['rendered']
    full_text = full_text + f"""
    Post title: {title}
    Post content: {content}
    """

In [24]:
full_text

'\n    Post title: Religion and the art of appearances\n    Post content: \n    \n    Post title: Enemies\n    Post content: \n<p>Blind faith, Authority that expects blind faith, and loved ones who enforce blind obedience to said authority, are all equally responsible for your predicament today.</p>\n\n\n\n<p>you love your parents, your parents love you. Or so they say. but they will do all kinds of evil things in the name of love, and not spend a minute thinking about what they are doing to you. I saw that message on reddit the other day &#8211; the usual gaslighting parent. You say your parents are not like this. Then why did you turn out this way? Parents are no special creatures. They the same frail old humans that all of us are. Why expect them to be better? In fact, a parent is expected to be worse, since they&#8217;re protecting something close to them. And we will do anything to preserve what&#8217;s going away from us. Parents are not to be trusted.</p>\n\n\n\n<p>And the same 

In [28]:
sys_2 = """You are an expert Social Media Marketer. You will be given the contents of the posts of a blog, and you must create an engaging Instagram Thread for the blog."""

prompt = f"""These are the posts. Create a nice instagram image caption for this.  {full_text}"""
prompt_2 = f"""These are the posts. Create a prompt to generate an image for this blog.  {full_text}"""

In [30]:
messages_for_full = [
    {"role":"system", "content": sys_2}, 
    {"role":"user", "content": prompt}
]
response_1 = client.chat.completions.create(
    model = 'gpt-4o', messages= messages_for_full
)

messages_for_im = [
    {"role":"system", "content": sys_2}, 
    {"role":"user", "content": prompt_2}
]
response_2 = client.chat.completions.create(
    model = 'gpt-4o', messages= messages_for_im
)

In [31]:
img_pp_2 = response_2.choices[0].message.content
image_response = client.images.generate(
  model="dall-e-3",
  prompt=img_pp_2,
  size="1024x1024",
  quality="standard",
  n=1,
  response_format='b64_json'
)

In [32]:
# Example usage
base64_string = image_response.data[0].b64_json
output_file = "output_image_2.jpg"

base64_to_jpg(base64_string, output_file)
print(f"Image saved as {output_file}")

Image saved as output_image_2.jpg


In [33]:
f_caption = response_1.choices[0].message.content 
with open('text_caption_full.txt', 'w') as f:
    f.write(f_caption)